# Import Libraries

In [5]:
import pickle
import numpy as np
import pandas as pd
import csv
import json
import os
import random
import shutil
from tqdm import tqdm
from pathlib import Path

from keras.utils import to_categorical
from keras import layers
from keras import models

import matplotlib.pyplot as plt
from numpy.random import seed
from keras.utils import plot_model

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



Documentation for VGG16  
https://keras.io/applications/  
https://keras.io/applications/#vgg16  


# Import and edit images

In [6]:

image_size = 64
train_dir = os.path.join("..","dataset","trainingset")
validation_dir = os.path.join("..","dataset","testset")

print("Train Dir: ",train_dir)
print("Validation Dir: ",validation_dir)

seed(4292)

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Train Dir:  ..\dataset\trainingset
Validation Dir:  ..\dataset\testset
Found 0 images belonging to 2 classes.


FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\dataset\\testset'

# Load the model

In [18]:
# create the base pre-trained model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

58892288/58889256 [==============================] - ETA: 12:0 - ETA: 8:0 - ETA: 7: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 56s - ETA: 55 - ETA: 52 - ETA: 51 - ETA: 50 - ETA: 49 - ETA: 46 - ETA: 45 - ETA: 44 - ETA: 42 - ETA: 41 - ETA: 39 - ETA: 38 - ETA: 37 - ETA: 36 - ETA: 34 - ETA: 34 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 28 - ETA: 29 - ETA: 28 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 22 - ETA

# Check and freeze the model

In [19]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000002743249C240> False
<keras.layers.convolutional.Conv2D object at 0x000002743249C588> False
<keras.layers.convolutional.Conv2D object at 0x000002743249C518> False
<keras.layers.pooling.MaxPooling2D object at 0x00000274324A64A8> False
<keras.layers.convolutional.Conv2D object at 0x00000274324A6320> False
<keras.layers.convolutional.Conv2D object at 0x00000274324C5860> False
<keras.layers.pooling.MaxPooling2D object at 0x00000274324DD278> False
<keras.layers.convolutional.Conv2D object at 0x00000274324DDF60> False
<keras.layers.convolutional.Conv2D object at 0x0000027432512278> False
<keras.layers.convolutional.Conv2D object at 0x000002743252A550> False
<keras.layers.pooling.MaxPooling2D object at 0x00000274325474A8> False
<keras.layers.convolutional.Conv2D object at 0x0000027432547F60> False
<keras.layers.convolutional.Conv2D object at 0x0000027432580278> False
<keras.layers.convolutional.Conv2D object at 0x0000027432599550> False
<ker

# Create my new model

In [20]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 4100      
Total params: 16,816,964
Trainable params: 9,181,700
Non-trainable params: 7,635,264
_________________________________________________________________


# Train the new model

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('./small_last4.h5')

# Check performance

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()


# See errors and images predicted

In [ ]:
# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()